# Code adapted from https://github.com/scjones5/nyt-bestsellers

In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 100)

In [2]:
r = requests.get("https://api.nytimes.com/svc/books/v3/lists/2019-08-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH")

In [3]:
bs = r.json()

In [4]:
for key in bs.keys():
    print(key)

status
copyright
num_results
last_modified
results


In [5]:
bs = json_normalize(bs["results"]["books"])

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_54512/577837607.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  bs = json_normalize(bs["results"]["books"])


In [6]:
bs.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,1538750554,9781538750551,Grand Central,A World War II veteran on parole must find the...,0.00,ONE GOOD DEED,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/One-Good-Deed-David-Bal...,,,,,,"[{'isbn10': '1538750562', 'isbn13': '978153875...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/f98ddab9-e895-565a-818a-b4488ec8c2e4
1,2,2,46,0,0,0735219095,9780735219090,Putnam,In a quiet town on the North Carolina coast in...,0.00,WHERE THE CRAWDADS SING,Delia Owens,by Delia Owens,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Where-Crawdads-Sing-Del...,,,,,,"[{'isbn10': '0735219095', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/053b4109-4555-5aa1-9b39-cc40549bcdf0
2,3,1,2,0,0,0062834894,9780062834898,Harper,"Gabriel Allon, the chief of Israeli intelligen...",0.00,THE NEW GIRL,Daniel Silva,by Daniel Silva,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/New-Girl-Novel-Gabriel-...,,,,,,"[{'isbn10': '0062834835', 'isbn13': '978006283...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/e308e25d-b1dd-559d-92b2-ec81b7bbad80
3,4,3,2,0,0,0385537077,9780385537070,Doubleday,Two boys respond to horrors at a Jim Crow-era ...,0.00,THE NICKEL BOYS,Colson Whitehead,by Colson Whitehead,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Nickel-Boys-Novel-Colso...,,https://www.nytimes.com/2019/07/11/books/revie...,,,,"[{'isbn10': '0385537077', 'isbn13': '978038553...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/b9c84bf7-ad1c-56a3-9b42-e9b37e488f42
4,5,8,24,0,0,0525620931,9780525620938,Doubleday,A decorated World War II veteran shoots and ki...,0.00,THE RECKONING,John Grisham,by John Grisham,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Reckoning-Novel-John-Gr...,,,,,,"[{'isbn10': '0385544154', 'isbn13': '978038554...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/821816fe-b672-571b-8cc7-5a23a25b58e8


In [7]:
bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   rank                  15 non-null     int64 
 1   rank_last_week        15 non-null     int64 
 2   weeks_on_list         15 non-null     int64 
 3   asterisk              15 non-null     int64 
 4   dagger                15 non-null     int64 
 5   primary_isbn10        15 non-null     object
 6   primary_isbn13        15 non-null     object
 7   publisher             15 non-null     object
 8   description           15 non-null     object
 9   price                 15 non-null     object
 10  title                 15 non-null     object
 11  author                15 non-null     object
 12  contributor           15 non-null     object
 13  contributor_note      15 non-null     object
 14  book_image            15 non-null     object
 15  book_image_width      15 non-null     int6

In [12]:
def parse_books(book_list, date):
    '''
    Based on: function parse_articles from
    This function takes in a response to the NYT books api and parses
    the books into a list of dictionaries
    '''
    books = []
    for i in book_list['results']['books']:
        dic = {}
        dic['age_group'] = i['age_group']
        dic['author'] = i['author'].encode("utf8")
        dic['book_review_link'] = i['book_review_link'].encode("utf8")
        dic['contributor'] = i['contributor'].encode("utf8")
        dic['dagger'] = i['dagger']
        dic['first_chapter_link'] = i['first_chapter_link'].encode("utf8")
        dic['price'] = i['price']
        dic['primary_isbn10'] = i['primary_isbn10'].encode("utf8")
        dic['primary_isbn13'] = i['primary_isbn13'].encode("utf8")
        dic['publisher'] = i['publisher'].encode("utf8")
        dic['sunday_review_link'] = i['sunday_review_link'].encode("utf8")
        dic['title'] = i['title'].encode("utf8")
        dic['weeks_on_list'] = i['weeks_on_list']
        dic['date'] = date  # add a date so that we know when it was a best-seller
        books.append(dic)
    return(books)


In [23]:
#sample request: https://api.nytimes.com/svc/books/v3/lists/2017-03-13/Combined%20Print%20and%20E-Book%20Fiction.json?api_key=<your_api_key>
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "combined-print-and-e-book-fiction"   #you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH"


bestsellers = []
weeks = []
num_calls_to_make = 312 # you can use up to 1000 api calls a day so max=1000
ctr = 0

# lists update once a week, every Sunday
for i in range(0,num_calls_to_make):
    today = datetime.datetime.today()
    idx = (today.weekday() + 1) % 7
    last_sun = today - datetime.timedelta((7*ctr)+idx)
    last_sun = last_sun.date()
    print(last_sun)
    url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, last_sun, LIST, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    if (r.status_code == 200):
        results = r.json()
        books = json_normalize(results["results"]["books"]) #parse_books(results, date)
        bestsellers.append(books)
        weeks.append(str(last_sun))
    else:
        print("API call did not work!")
        print(r)
        #break
    ctr = ctr + 1
    time.sleep(6)
#     if (bestsellers[0].keys() is not None):
#        keys = bestsellers[0].keys()
#        # Print all results to a csv file
#        with open('bestsellers.csv', 'wb') as output_file:
#            dict_writer = csv.DictWriter(output_file, keys)
#            dict_writer.writeheader()
#            dict_writer.writerows(bestsellers)
bestsellers_df = pd.concat(bestsellers)

2022-06-05
https://api.nytimes.com/svc/books/v3/lists/2022-06-05/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH


/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_54512/2016006589.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  books = json_normalize(results["results"]["books"]) #parse_books(results, date)


2022-05-29
https://api.nytimes.com/svc/books/v3/lists/2022-05-29/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-22
https://api.nytimes.com/svc/books/v3/lists/2022-05-22/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-15
https://api.nytimes.com/svc/books/v3/lists/2022-05-15/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-08
https://api.nytimes.com/svc/books/v3/lists/2022-05-08/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-05-01
https://api.nytimes.com/svc/books/v3/lists/2022-05-01/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-24
https://api.nytimes.com/svc/books/v3/lists/2022-04-24/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-17
https://api.nytimes.com/svc/books/v3/lists/2022-04-17/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2022-04-10
https://a

2021-04-11
https://api.nytimes.com/svc/books/v3/lists/2021-04-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-04-04
https://api.nytimes.com/svc/books/v3/lists/2021-04-04/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-28
https://api.nytimes.com/svc/books/v3/lists/2021-03-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-21
https://api.nytimes.com/svc/books/v3/lists/2021-03-21/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-14
https://api.nytimes.com/svc/books/v3/lists/2021-03-14/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-03-07
https://api.nytimes.com/svc/books/v3/lists/2021-03-07/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-28
https://api.nytimes.com/svc/books/v3/lists/2021-02-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2021-02-21
https://a

2020-02-23
https://api.nytimes.com/svc/books/v3/lists/2020-02-23/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-16
https://api.nytimes.com/svc/books/v3/lists/2020-02-16/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-09
https://api.nytimes.com/svc/books/v3/lists/2020-02-09/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-02-02
https://api.nytimes.com/svc/books/v3/lists/2020-02-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-26
https://api.nytimes.com/svc/books/v3/lists/2020-01-26/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-19
https://api.nytimes.com/svc/books/v3/lists/2020-01-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-12
https://api.nytimes.com/svc/books/v3/lists/2020-01-12/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2020-01-05
https://a

2019-01-06
https://api.nytimes.com/svc/books/v3/lists/2019-01-06/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-12-30
https://api.nytimes.com/svc/books/v3/lists/2018-12-30/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-12-23
https://api.nytimes.com/svc/books/v3/lists/2018-12-23/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-12-16
https://api.nytimes.com/svc/books/v3/lists/2018-12-16/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-12-09
https://api.nytimes.com/svc/books/v3/lists/2018-12-09/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-12-02
https://api.nytimes.com/svc/books/v3/lists/2018-12-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-11-25
https://api.nytimes.com/svc/books/v3/lists/2018-11-25/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2018-11-18
https://a

2017-11-19
https://api.nytimes.com/svc/books/v3/lists/2017-11-19/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-11-12
https://api.nytimes.com/svc/books/v3/lists/2017-11-12/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-11-05
https://api.nytimes.com/svc/books/v3/lists/2017-11-05/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-10-29
https://api.nytimes.com/svc/books/v3/lists/2017-10-29/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-10-22
https://api.nytimes.com/svc/books/v3/lists/2017-10-22/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-10-15
https://api.nytimes.com/svc/books/v3/lists/2017-10-15/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-10-08
https://api.nytimes.com/svc/books/v3/lists/2017-10-08/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2017-10-01
https://a

2016-10-02
https://api.nytimes.com/svc/books/v3/lists/2016-10-02/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-25
https://api.nytimes.com/svc/books/v3/lists/2016-09-25/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-18
https://api.nytimes.com/svc/books/v3/lists/2016-09-18/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-11
https://api.nytimes.com/svc/books/v3/lists/2016-09-11/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-09-04
https://api.nytimes.com/svc/books/v3/lists/2016-09-04/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-08-28
https://api.nytimes.com/svc/books/v3/lists/2016-08-28/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-08-21
https://api.nytimes.com/svc/books/v3/lists/2016-08-21/combined-print-and-e-book-fiction?api-key=LarCbQ96wJoZWk2SH42j8MZhCgAMLGwH
2016-08-14
https://a

In [24]:
weeks

['2022-06-05',
 '2022-05-29',
 '2022-05-22',
 '2022-05-15',
 '2022-05-08',
 '2022-05-01',
 '2022-04-24',
 '2022-04-17',
 '2022-04-10',
 '2022-04-03',
 '2022-03-27',
 '2022-03-20',
 '2022-03-13',
 '2022-03-06',
 '2022-02-27',
 '2022-02-20',
 '2022-02-13',
 '2022-02-06',
 '2022-01-30',
 '2022-01-23',
 '2022-01-16',
 '2022-01-09',
 '2022-01-02',
 '2021-12-26',
 '2021-12-19',
 '2021-12-12',
 '2021-12-05',
 '2021-11-28',
 '2021-11-21',
 '2021-11-14',
 '2021-11-07',
 '2021-10-31',
 '2021-10-24',
 '2021-10-17',
 '2021-10-10',
 '2021-10-03',
 '2021-09-26',
 '2021-09-19',
 '2021-09-12',
 '2021-09-05',
 '2021-08-29',
 '2021-08-22',
 '2021-08-15',
 '2021-08-08',
 '2021-08-01',
 '2021-07-25',
 '2021-07-18',
 '2021-07-11',
 '2021-07-04',
 '2021-06-27',
 '2021-06-20',
 '2021-06-13',
 '2021-06-06',
 '2021-05-30',
 '2021-05-23',
 '2021-05-16',
 '2021-05-09',
 '2021-05-02',
 '2021-04-25',
 '2021-04-18',
 '2021-04-11',
 '2021-04-04',
 '2021-03-28',
 '2021-03-21',
 '2021-03-14',
 '2021-03-07',
 '2021-02-

In [25]:
bestsellers_df.to_csv("../data/last_six_years_nyt_bestsellers.csv")

In [26]:
bestsellers_df.head(50)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,1982181656,9781982181659,Atria/Emily Bestler,The fifth book in the Terminal List series. Ja...,0.00,IN THE BLOOD,Jack Carr,by Jack Carr,,https://storage.googleapis.com/du-prd/books/im...,337,500,https://www.amazon.com/dp/1982181656?tag=NYTBS...,,,,,,"[{'isbn10': '1982181656', 'isbn13': '978198218...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/15be5903-78f0-5d2a-9ca7-f0e468dab5c5
1,2,1,3,0,0,0593334833,9780593334836,Berkley,"While on vacation in North Carolina, a literar...",0.00,BOOK LOVERS,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,332,500,https://www.amazon.com/dp/0593334833?tag=NYTBS...,,,,,,"[{'isbn10': '0593334833', 'isbn13': '978059333...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/31fc527e-86c7-5743-a111-ea333a630706
2,3,2,49,0,0,1501110365,9781501110368,Atria,A battered wife raised in a violent home attem...,0.00,IT ENDS WITH US,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,319,495,http://www.amazon.com/Ends-Us-Novel-Colleen-Ho...,,,,,,"[{'isbn10': '1501110365', 'isbn13': '978150111...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e2a3545e-e9cb-5828-9d97-50a798a0e4f6
3,4,5,161,0,0,0735219109,9780735219106,Putnam,In a quiet town on the North Carolina coast in...,0.00,WHERE THE CRAWDADS SING,Delia Owens,by Delia Owens,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Where-Crawdads-Sing-Del...,,,,,,"[{'isbn10': '0735219095', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/08a5730d-43e5-586d-8ce6-4b891eb75fe1
4,5,6,24,0,0,1538724731,9781538724736,Grand Central,Lowen Ashleigh is hired by the husband of an i...,0.00,VERITY,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,324,500,https://www.amazon.com/dp/1791392792?tag=NYTBS...,,,,,,"[{'isbn10': '1791392792', 'isbn13': '978179139...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/013c7729-a1de-5e5c-921a-da43243f9a4a
5,6,7,47,0,0,1501161938,9781501161933,Washington Square/Atria,A movie icon recounts stories of her loves and...,0.00,THE SEVEN HUSBANDS OF EVELYN HUGO,Taylor Jenkins Reid,by Taylor Jenkins Reid,,https://storage.googleapis.com/du-prd/books/im...,315,500,https://www.amazon.com/dp/1501161938?tag=NYTBS...,,,,,,"[{'isbn10': '1501161938', 'isbn13': '978150116...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/d9511fac-ee44-5a87-9af7-2cd6a6f8f984
6,7,9,19,0,0,1476753180,9781476753188,Atria,"Tate Collins and Miles Archer, an airline pilo...",0.00,UGLY LOVE,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,128,198,http://www.amazon.com/Ugly-Love-Novel-Colleen-...,,,,,,"[{'isbn10': '1476753180', 'isbn13': '978147675...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/08464a1b-cfd4-5d1b-8898-e206fb95deeb
7,8,3,3,0,0,0316499374,9780316499378,"Little, Brown",The 22nd book in the Women’s Murder Club serie...,0.00,22 SECONDS,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,323,500,https://www.amazon.com/dp/0316499374?tag=NYTBS...,,,,,,"[{'isbn10': '0316499374', 'isbn13': '978031649...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/30e66d4a-611a-5a29-b8e2-c7b2eed2bbd4
8,9,0,1,0,0,052553900X,9780525539001,Riverhead,A 40-year-old woman finds new meaning in past ...,0.00,THIS TIME TOMORROW,Emma Straub,by Emma Straub,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/052553900X?tag=NYTBS...,,,,,,"[{'isbn10': '05255

In [18]:
print(last_sun)

2022-06-05
